[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [1]:
import torch

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [2]:
D_in, H, D_out = 54, 100, 7

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# D_in, H1, H2, D_out = 784, 100, 50, 10
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H1),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H1, H2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H2, D_out),
# )


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [3]:
outputs = model(torch.randn(600, 54))
outputs.shape

torch.Size([600, 7])

In [4]:
print(outputs[0][:])

tensor([-0.0340, -0.4522,  0.1037, -0.0883, -0.0435, -0.0978,  0.0092],
       grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [5]:
model

Sequential(
  (0): Linear(in_features=54, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=7, bias=True)
)

In [6]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=54, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=7, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [7]:
import pandas as pd
from sklearn.utils import Bunch

# Cargar dataset desde tu CSV
df = pd.read_csv("/content/covtype.csv")

# Separar features y target
X = df.drop("Cover_Type", axis=1)
Y = df["Cover_Type"]

# Crear un objeto tipo Bunch como el de fetch_openml
covtype = Bunch(data=X, target=Y)

# Ahora puedes usarlo igual que MNIST
X, Y = covtype["data"], covtype["target"]

print(X.shape, Y.shape)

(581012, 54) (581012,)


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Pasar a numpy arrays
x_2 = np.array(X, dtype=np.float32)
y_2 = np.array(Y, dtype=np.int32)

# Normalización (media=0, varianza=1)
scaler = StandardScaler()
x_2 = scaler.fit_transform(x_2)

# Split 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    x_2, y_2, test_size=0.2, random_state=42, stratify=y_2
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(464809, 54) (116203, 54) (464809,) (116203,)


In [9]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
# X_train

In [10]:
torch.cuda.is_available()

True

In [11]:
print(X)

        Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0            2596      51      3                               258   
1            2590      56      2                               212   
2            2804     139      9                               268   
3            2785     155     18                               242   
4            2595      45      2                               153   
...           ...     ...    ...                               ...   
581007       2396     153     20                                85   
581008       2391     152     19                                67   
581009       2386     159     17                                60   
581010       2384     170     15                                60   
581011       2383     165     13                                60   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                                    0                              510   
1        

In [14]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy((y_train - 1).astype(np.int64)).long().cuda()

# bucle entrenamiento
epochs = 350
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/350 Loss nan
Epoch 20/350 Loss nan
Epoch 30/350 Loss nan
Epoch 40/350 Loss nan
Epoch 50/350 Loss nan
Epoch 60/350 Loss nan
Epoch 70/350 Loss nan
Epoch 80/350 Loss nan
Epoch 90/350 Loss nan
Epoch 100/350 Loss nan
Epoch 110/350 Loss nan
Epoch 120/350 Loss nan
Epoch 130/350 Loss nan
Epoch 140/350 Loss nan
Epoch 150/350 Loss nan
Epoch 160/350 Loss nan
Epoch 170/350 Loss nan
Epoch 180/350 Loss nan
Epoch 190/350 Loss nan
Epoch 200/350 Loss nan
Epoch 210/350 Loss nan
Epoch 220/350 Loss nan
Epoch 230/350 Loss nan
Epoch 240/350 Loss nan
Epoch 250/350 Loss nan
Epoch 260/350 Loss nan
Epoch 270/350 Loss nan
Epoch 280/350 Loss nan
Epoch 290/350 Loss nan
Epoch 300/350 Loss nan
Epoch 310/350 Loss nan
Epoch 320/350 Loss nan
Epoch 330/350 Loss nan
Epoch 340/350 Loss nan
Epoch 350/350 Loss nan


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [15]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    with torch.no_grad():
      y_pred = model(x)
      y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

# Convertir X_test a tensor en GPU
X_test_t = torch.from_numpy(X_test).float().cuda()

# Corregir etiquetas (1–7 → 0–6)
y_test_fixed = (y_test - 1).astype(np.int64)

# Predicciones
y_pred = evaluate(X_test_t)

# Accuracy
acc = accuracy_score(y_test_fixed, y_pred.cpu().numpy())
print(f"Accuracy en test: {acc:.4f}")

Accuracy en test: 0.3646


Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [16]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [18]:
D_in, H, D_out = 54, 100, 7

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Y_t = torch.from_numpy((y_train - 1).astype(np.int64)).long().cuda()

X_t = torch.from_numpy(X_train).float().cuda()

epochs = 1500
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/1500 Loss 1.16789
Epoch 20/1500 Loss 1.01057
Epoch 30/1500 Loss 0.93038
Epoch 40/1500 Loss 0.88197
Epoch 50/1500 Loss 0.84722
Epoch 60/1500 Loss 0.82145
Epoch 70/1500 Loss 0.80136
Epoch 80/1500 Loss 0.78505
Epoch 90/1500 Loss 0.77150
Epoch 100/1500 Loss 0.76002
Epoch 110/1500 Loss 0.75010
Epoch 120/1500 Loss 0.74140
Epoch 130/1500 Loss 0.73367
Epoch 140/1500 Loss 0.72673
Epoch 150/1500 Loss 0.72046
Epoch 160/1500 Loss 0.71475
Epoch 170/1500 Loss 0.70951
Epoch 180/1500 Loss 0.70468
Epoch 190/1500 Loss 0.70020
Epoch 200/1500 Loss 0.69604
Epoch 210/1500 Loss 0.69216
Epoch 220/1500 Loss 0.68851
Epoch 230/1500 Loss 0.68508
Epoch 240/1500 Loss 0.68184
Epoch 250/1500 Loss 0.67877
Epoch 260/1500 Loss 0.67586
Epoch 270/1500 Loss 0.67308
Epoch 280/1500 Loss 0.67044
Epoch 290/1500 Loss 0.66791
Epoch 300/1500 Loss 0.66549
Epoch 310/1500 Loss 0.66317
Epoch 320/1500 Loss 0.66094
Epoch 330/1500 Loss 0.65880
Epoch 340/1500 Loss 0.65674
Epoch 350/1500 Loss 0.65476
Epoch 360/1500 Loss 0.65286
E

0.04992986411710541

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [19]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [20]:
model = ModeloPersonalizado(54, 100, 7)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(500, 54)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.7781, -0.4585, -1.5557,  ..., -0.1267, -1.5205,  0.2482],
        [ 0.6791, -1.3579, -0.2597,  ..., -0.6724, -0.5778, -1.3143],
        [ 1.8858,  0.0670,  1.1524,  ..., -0.9702, -0.1827, -0.6335],
        ...,
        [-0.6763,  1.1758,  1.7239,  ..., -0.7012,  0.7725,  0.0133],
        [ 0.5859,  2.0100,  0.9898,  ...,  0.9272, -1.9069,  0.7469],
        [-0.4950, -0.2086,  0.2589,  ..., -0.2553,  1.1328,  0.0307]])


torch.Size([500, 7])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [21]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy((y_train - 1).astype(np.int64)).long().cuda()

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.11708
Epoch 20/100 Loss 0.97090
Epoch 30/100 Loss 0.90646
Epoch 40/100 Loss 0.86141
Epoch 50/100 Loss 0.83152
Epoch 60/100 Loss 0.80865
Epoch 70/100 Loss 0.79048
Epoch 80/100 Loss 0.77584
Epoch 90/100 Loss 0.76367
Epoch 100/100 Loss 0.75325


0.07461941602196157

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [22]:
class ModelCustom2(torch.nn.Module):

    def __init__(self, D_in, H, D_out):
        super(ModelCustom2, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [23]:
model = ModelCustom2(54, 100, 7).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy((y_train - 1).astype(np.int64)).long().cuda()

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.20897
Epoch 20/100 Loss 0.99608
Epoch 30/100 Loss 0.90713
Epoch 40/100 Loss 0.85752
Epoch 50/100 Loss 0.82543
Epoch 60/100 Loss 0.80270
Epoch 70/100 Loss 0.78559
Epoch 80/100 Loss 0.77211
Epoch 90/100 Loss 0.76113
Epoch 100/100 Loss 0.75194


0.1123034689293736

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [24]:
model

ModelCustom2(
  (fc1): Linear(in_features=54, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=7, bias=True)
)

In [25]:
model.fc1

Linear(in_features=54, out_features=100, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [26]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0031,  0.0714,  0.0770,  ...,  0.0239, -0.0288, -0.0630],
        [ 0.0244,  0.0344,  0.0729,  ..., -0.0571,  0.0500,  0.0279],
        [-0.1135, -0.0349, -0.0513,  ..., -0.1303, -0.0727, -0.1166],
        ...,
        [-0.2517, -0.1013,  0.1205,  ..., -0.0147, -0.1496,  0.1100],
        [-0.0465, -0.1048,  0.1291,  ...,  0.0529, -0.1210, -0.0795],
        [ 0.1224, -0.0613,  0.1408,  ...,  0.0046, -0.0835,  0.0087]],
       device='cuda:0', requires_grad=True)

In [27]:
model.fc1.bias

Parameter containing:
tensor([ 5.0099e-02, -3.9974e-02,  5.8869e-02,  9.4687e-02,  1.8843e-02,
        -4.0517e-02,  2.7967e-01,  6.3748e-02,  1.3238e-01,  9.1496e-02,
         1.3607e-01,  4.7311e-02,  1.0187e-01, -8.4626e-02,  2.3545e-01,
         1.6373e-01,  3.5412e-02, -1.0690e-01,  4.2924e-02, -7.6842e-02,
         1.0524e-02,  7.0393e-02,  3.1519e-01,  1.0598e-01,  1.2837e-01,
        -1.7713e-02,  5.5963e-02,  2.4937e-02,  9.8665e-02,  1.7711e-01,
         5.4843e-02, -6.4687e-02,  8.3909e-02, -6.1571e-02, -7.9759e-02,
         1.0703e-01,  1.2323e-01,  3.0121e-01,  1.2234e-01, -7.2656e-02,
         6.2813e-02,  3.4614e-02,  1.8632e-01, -1.4641e-01, -6.1977e-02,
        -3.9726e-03,  5.5159e-03,  8.5251e-02,  1.7911e-01,  2.4912e-02,
        -1.1540e-02, -5.4404e-02, -2.5532e-02,  2.1383e-01,  1.1876e-01,
         9.4801e-02,  2.0144e-01,  1.0957e-01,  1.4105e-03,  6.6497e-02,
         4.7184e-02, -2.4171e-02, -9.5881e-03, -1.0617e-01, -5.9555e-02,
        -1.7026e-01, -3.6401e

Es posible sobreescribir una capa de la siguiente manera

In [28]:
model.fc2 = torch.nn.Linear(100, 1)

model

ModelCustom2(
  (fc1): Linear(in_features=54, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [29]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=54, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [30]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=54, out_features=100, bias=True)
)

In [31]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=54, out_features=100, bias=True)
  (1): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes.